# ViT FineTuning with CIFAR-10 

## Import Library

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from datasets import load_dataset,load_metric

from transformers import ViTFeatureExtractor,ViTForImageClassification,TrainingArguments, Trainer

## Load Dataset (CIFAR10)

In [2]:
dataset = load_dataset('cifar10',split='train')
dataset

Found cached dataset cifar10 (C:/Users/USER/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)


Dataset({
    features: ['img', 'label'],
    num_rows: 50000
})

In [3]:
label2id = {"plane":0, "car":1, "bird":2, "cat":3, "deer":4, "dog":5, "frog":6, "horse":7, "ship":8, "truck":9}
id2label = {0:"plane", 1:"car", 2:"bird", 3:"cat", 4:"deer", 5:"dog", 6:"frog", 7:"horse", 8:"ship", 9:"truck"}

In [4]:
model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

In [5]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['img']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['label'] = example_batch['label']
    return inputs

In [6]:
prepared_ds = dataset.with_transform(transform)
prepared_ds = prepared_ds.train_test_split(test_size=0.1)
prepared_ds

DatasetDict({
    train: Dataset({
        features: ['img', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['img', 'label'],
        num_rows: 5000
    })
})

## Training

In [7]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [8]:
metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [9]:
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=10,
    id2label=id2label,
    label2id=label2id)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
  output_dir="./",
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=2,
  fp16=True,
  save_steps=10000,
  eval_steps=10000,
  logging_steps=100,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  
)

PyTorch: setting up devices


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["test"],
    tokenizer=feature_extractor,
)

Using cuda_amp half precision backend


In [12]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

***** Running training *****
  Num examples = 45000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 11250
  Number of trainable parameters = 85806346
  1%|          | 100/11250 [03:08<5:46:19,  1.86s/it]

{'loss': 2.191, 'learning_rate': 0.00019873777777777778, 'epoch': 0.02}


  2%|▏         | 200/11250 [06:16<5:45:58,  1.88s/it]

{'loss': 1.7366, 'learning_rate': 0.00019696, 'epoch': 0.04}


  3%|▎         | 300/11250 [09:24<5:42:42,  1.88s/it]

...

{'loss': 0.1743, 'learning_rate': 1.5288888888888889e-06, 'epoch': 1.99}


100%|██████████| 11250/11250 [5:50:37<00:00,  2.06s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./checkpoint-10000 (score: 0.1873820275068283).
100%|██████████| 11250/11250 [5:50:37<00:00,  1.87s/it]
Saving model checkpoint to ./
Configuration saved in ./config.json


{'train_runtime': 21037.8953, 'train_samples_per_second': 4.278, 'train_steps_per_second': 0.535, 'train_loss': 0.25218826421101886, 'epoch': 2.0}


Model weights saved in ./pytorch_model.bin
Image processor saved in ./preprocessor_config.json


***** train metrics *****
  epoch                    =        2.0
  train_loss               =     0.2522
  train_runtime            = 5:50:37.89
  train_samples_per_second =      4.278
  train_steps_per_second   =      0.535


In [13]:
dataset_test = load_dataset('cifar10',split='test')
dataset_test

Found cached dataset cifar10 (C:/Users/USER/.cache/huggingface/datasets/cifar10/plain_text/1.0.0/447d6ec4733dddd1ce3bb577c7166b986eaa4c538dcd9e805ba61f35674a9de4)


Dataset({
    features: ['img', 'label'],
    num_rows: 10000
})

## Testing

In [21]:
model = ViTForImageClassification.from_pretrained(
    './checkpoint-10000/',
    num_labels=10,
    id2label=id2label,
    label2id=label2id)

loading configuration file ./checkpoint-10000/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "plane",
    "1": "car",
    "2": "bird",
    "3": "cat",
    "4": "deer",
    "5": "dog",
    "6": "frog",
    "7": "horse",
    "8": "ship",
    "9": "truck"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "bird": 2,
    "car": 1,
    "cat": 3,
    "deer": 4,
    "dog": 5,
    "frog": 6,
    "horse": 7,
    "plane": 0,
    "ship": 8,
    "truck": 9
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "problem_type": "single_label_classification",
  "qkv_bias": true,
  "torch_

In [23]:
prepared_ds_test = dataset_test.with_transform(transform)

In [24]:
# Predictions
y_test_predict = trainer.predict(prepared_ds_test)

# Take a look at the predictions
y_test_predict

***** Running Prediction *****
  Num examples = 10000
  Batch size = 8
100%|██████████| 1250/1250 [14:25<00:00,  1.44it/s]


PredictionOutput(predictions=array([[ 2.65  ,  1.558 ,  1.853 , ...,  2.688 ,  1.09  ,  1.026 ],
       [ 3.559 ,  4.49  ,  1.596 , ...,  1.493 , 12.81  ,  2.729 ],
       [ 4.184 ,  3.684 ,  1.752 , ...,  1.954 , 10.734 ,  2.432 ],
       ...,
       [ 1.27  , -0.089 ,  1.393 , ...,  2.295 ,  0.6973,  1.25  ],
       [ 6.598 ,  7.43  ,  2.408 , ...,  2.807 ,  3.254 ,  3.709 ],
       [ 1.143 ,  1.122 ,  2.422 , ..., 11.66  ,  1.349 ,  1.61  ]],
      dtype=float16), label_ids=array([3, 8, 8, ..., 5, 1, 7], dtype=int64), metrics={'test_loss': 0.20327411592006683, 'test_accuracy': 0.9399, 'test_runtime': 866.4282, 'test_samples_per_second': 11.542, 'test_steps_per_second': 1.443})